In [ ]:
!pip install langchain langchain-openai openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 7.8 MB/s eta 0:00:00


In [ ]:
from getpass import getpass

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

Введите ваш OpenAI API ключ··········


## Если используете ключ из курса, запустите эти ячейки 👇

In [ ]:
from langchain_openai import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш API ключ, полученный в боте курса··········


💾 Добавим памяти LLM'ке! 🦙







In [ ]:
# подгрузим датасет
import pandas as pd

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1084404/dial_df.csv")
df.head()

,dialogue_id,question
0,1,Под каким номером играл Криштиану Роналду в Юв...
1,1,Под каким номером он играл в Манчестер Юнайтед?
2,1,В каком году он был рожден?
3,2,Сколько лет прожил величайший русский поэт Але...
4,2,В каком году он родился?


In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Создадим словарь, который будет мапить session_id с историей диалога этой сессии
store = {}

# Напишем функцию, которая возвращает историю диалога по session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [ ]:
### ваше решение
# Создаём конфиг для нашего Runnable, чтобы указать ему session_id при вызове
config = {"configurable": {"session_id": "1"}}

llm_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history
)


'Привет, Иван! У меня всё хорошо, спасибо! Как у тебя дела? Чем могу помочь?'

In [17]:
# ответьте на вопросы с помощью ЛЛМки, не забывайте очищать историю диалога при смене темы
ans = []
sess = None
prev_sess = None
for id, q in zip(df['dialogue_id'], df['question']):
    sess = id
    if sess != prev_sess:
      prev_sess = sess
      get_session_history('1').clear()
    config = {"configurable": {"session_id": id}}
    answer = llm_with_history.invoke("Выводи только цифру числом. Больше ничего" + q, config=config).content
    ans.append(answer) # здесь инференс ЛЛМки
    #if
    #break # уберем break, когда убедимся, что работает на одном примере

In [18]:
ans

['7',
 '7',
 '1985',
 '37',
 '1799',
 '1837',
 '4',
 '1940',
 '1970',
 '1879',
 '1955',
 '76',
 '4',
 '1962',
 '46',
 '38',
 '14',
 '8848']

In [20]:
df['answer'] = ans # запишем предсказания в датафрейм
df.to_csv('solution.csv', index=False) # сохраним решение в csv

In [21]:
df

,dialogue_id,question,answer
0,1,Под каким номером играл Криштиану Роналду в Юв...,7
1,1,Под каким номером он играл в Манчестер Юнайтед?,7
2,1,В каком году он был рожден?,1985
3,2,Сколько лет прожил величайший русский поэт Але...,37
4,2,В каком году он родился?,1799
5,2,Дуэль в каком году стала для него последней?,1837
6,3,Сколько было участников в золотом составе груп...,4
7,3,В каком году родился барабанщик золотого соста...,1940
8,3,В каком году распалась группа?,1970
9,4,В каком году родился Альберт Эйнштейн?,1879
